In [15]:
import sys
import os 
from pathlib import Path
import pandas as pd
import json

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

from scripts.utils import log_df

In [30]:
openngc_types_path = project_path / 'raw_data' / 'OpenNGC_types.csv' 

stellarium_path = project_path / 'data' / 'draft'/'observingList_DwarfII.json' 
stellarium_types_path = project_path / 'raw_data' / 'stellarium_types.csv' 
moon_planets_path = project_path / 'raw_data' / 'moon_planets.csv' 

 

## create stellarium types csv

In [18]:
with open(stellarium_path, 'r') as f:
    data = json.load(f)

In [4]:
objects = data['observingLists']['observingLists']['{5e727f81-e0a8-43f0-9258-3848aa2d9762}']['objects']

records = []
tmp = set()
for object in objects:
    if object['objtype'] not in tmp:
        records.append({'objtype': object['objtype'], 'type': object['type'], 'category': ''})
        tmp.add(object['objtype'])

In [5]:
        
df = pd.DataFrame(records)
log_df(df)

(13, 3)


,objtype,type,category
0,globular star cluster,Nebula,
1,planet,Planet,
2,galaxy,Nebula,
3,HII region,Nebula,
4,open star cluster,Nebula,


In [54]:
df.to_csv(stellarium_types_path, index=False)

## create type / type category dictionary

In [25]:
grouped_types = {}
types = {}

In [26]:
stellarium_df = pd.read_csv(stellarium_types_path)

log_df(stellarium_df)

(13, 3)


,objtype,type,category
0,globular star cluster,Nebula,clusters
1,planet,Planet,moon_planets
2,galaxy,Nebula,galaxies
3,HII region,Nebula,nebulae
4,open star cluster,Nebula,clusters


In [27]:
openngc_df = pd.read_csv(openngc_types_path)

log_df(openngc_df)

(21, 3)


,code,name,category
0,*,Star,stars
1,**,Double star,stars
2,*Ass,Association of stars,stars
3,OCl,Open Cluster,clusters
4,GCl,Globular Cluster,clusters


In [34]:
planets_df = pd.read_csv(moon_planets_path)

log_df(planets_df)

(8, 9)


,Apparent magnitude (V),Object,Name,Seen from...,Notes,type,category,catalog,order
0,-3.69,Moon,Moon,"lit by earthlight, reflecting earthshine seen ...",NaN,Moon,moon_planets,-,0
1,0.23,planet Mercury,Mercury,seen from Earth,mean brightness[32],Planet,moon_planets,-,1
2,-4.14,planet Venus,Venus,seen from Earth,mean brightness[32],Planet,moon_planets,-,2
3,0.71,planet Mars,Mars,seen from Earth,mean brightness[32],Planet,moon_planets,-,4
4,-2.20,planet Jupiter,Jupiter,seen from Earth,mean brightness[32],Planet,moon_planets,-,5


In [36]:
for index, rows in openngc_df.iterrows():
    types[rows['name']] = rows['category']

for index, rows in stellarium_df.iterrows():
    types[rows['objtype']] = rows['category']

for index, rows in planets_df.iterrows():
    types[rows['type']] = rows['category']

dict(sorted(types.items()))

{'Association of stars': 'stars',
 'Dark Nebula': 'nebulae',
 'Double star': 'stars',
 'Duplicated object (see NGC or IC columns to find the master object)': 'duplicate',
 'Emission Nebula': 'nebulae',
 'Galaxy': 'galaxies',
 'Galaxy Pair': 'galaxies',
 'Galaxy Triplet': 'galaxies',
 'Globular Cluster': 'clusters',
 'Group of galaxies': 'galaxies',
 'HII Ionized region': 'nebulae',
 'HII region': 'nebulae',
 'Moon': 'moon_planets',
 'Nebula': 'nebulae',
 'Nonexistent object': 'nonexistent',
 'Nova star': 'stars',
 'Open Cluster': 'clusters',
 'Other classification (see object notes)': 'other',
 'Planet': 'moon_planets',
 'Planetary Nebula': 'nebulae',
 'Reflection Nebula': 'nebulae',
 'Star': 'stars',
 'Star cluster + Nebula': 'nebulae',
 'Supernova remnant': 'nebulae',
 'cluster associated with nebulosity': 'nebulae',
 'double star': 'stars',
 'double star, pulsating variable star': 'stars',
 'emission nebula': 'nebulae',
 'galaxy': 'galaxies',
 'globular star cluster': 'clusters',
 '

In [39]:
for index, rows in openngc_df.iterrows():
    category = rows['category']
    name = rows['name']
    if category not in grouped_types:
        grouped_types[category] = []

    grouped_types[category].append(name)

for index, rows in stellarium_df.iterrows():
    category = rows['category']
    name = rows['objtype']
    if category not in grouped_types:
        grouped_types[category] = []

    grouped_types[category].append(name)
    

grouped_types

{'stars': ['Star',
  'Double star',
  'Association of stars',
  'Nova star',
  'star',
  'double star, pulsating variable star',
  'double star',
  'Star',
  'Double star',
  'Association of stars',
  'Nova star',
  'star',
  'double star, pulsating variable star',
  'double star'],
 'clusters': ['Open Cluster',
  'Globular Cluster',
  'globular star cluster',
  'open star cluster',
  'Open Cluster',
  'Globular Cluster',
  'globular star cluster',
  'open star cluster'],
 'nebulae': ['Star cluster + Nebula',
  'Planetary Nebula',
  'HII Ionized region',
  'Dark Nebula',
  'Emission Nebula',
  'Nebula',
  'Reflection Nebula',
  'Supernova remnant',
  'HII region',
  'supernova remnant',
  'cluster associated with nebulosity',
  'planetary nebula',
  'emission nebula',
  'Star cluster + Nebula',
  'Planetary Nebula',
  'HII Ionized region',
  'Dark Nebula',
  'Emission Nebula',
  'Nebula',
  'Reflection Nebula',
  'Supernova remnant',
  'HII region',
  'supernova remnant',
  'cluster as